In [ ]:
!unzip 'poetry.zip'

Archive:  poetry.zip
   creating: poetry/
  inflating: poetry/poetry_data_train — копия.json  
  inflating: __MACOSX/poetry/._poetry_data_train — копия.json  
  inflating: poetry/poetry_data_test — копия.json  
  inflating: __MACOSX/poetry/._poetry_data_test — копия.json  


In [ ]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 3.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

TEST_DATA_PATH = '/content/poetry/poetry_data_test — копия.json'
TRAIN_DATA_PATH = '/content/poetry/poetry_data_train — копия.json'

test_data = pd.read_json(TEST_DATA_PATH)
train_data = pd.read_json(TRAIN_DATA_PATH)

cuda


In [ ]:
test_data.head()

,url,rating,views,output_text,genre
0,https://www.chitalnya.ru/work/2999707/,0,15,"Нынче головы четыре у дракона,\nПожиратель пре...",лирика
1,https://www.chitalnya.ru/work/1026761/,0,167,Сало Украине! Героям - Сало!\nЭтого продукта-д...,юмор
2,https://www.chitalnya.ru/work/2260544/,7,62,"Стая птиц летит по небосводу,\nЗа собою, в про...",лирика
3,https://www.chitalnya.ru/work/1889307/,9,132,"Холодно в дому, мысли как в сиропе,\nДаже свет...",лирика
4,https://www.chitalnya.ru/work/788355/,0,195,Иду я на вы!\nСвятослав говорил.\nЛежу я на вы...,юмор


## Baseline

In [ ]:
# заменяем рейтинг на бинарную переменную 0/не0
# тексты - lower
# жанры - lower
# удаляем ссылки

def preprocess_data(df):
  df_mod = df.copy()
  df_mod.output_text = df_mod.output_text.str.lower()
  df_mod.rating = df_mod.rating.apply(lambda x: 1 if x > 0 else 0)

  return df_mod[['output_text', 'rating']]


train_data_mod = preprocess_data(train_data)
test_data_mod = preprocess_data(test_data)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data_mod.output_text)
X_test = vectorizer.transform(test_data_mod.output_text)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42).fit(X_train, train_data_mod.rating)

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score

roc_auc_score(test_data_mod.rating, clf.predict_proba(X_test)[:, 1]), accuracy_score(test_data_mod.rating, clf.predict(X_test))

(0.6518974744622479, 0.6201335298708055)

## BERT

In [ ]:
# заменяем рейтинг на бинарную переменную 0/не0
# для стихов может быть важно сохранить структуру (переносы строк), поэтому добавим новый токен для обозначения переносов строк + lower
# жанры приведем к нижнему регистру
# удаляем ссылки

NEWLINE_SPEC_TOKEN = '[NEWLINE]'

def preprocess_data(df):
  df_mod = df.copy()
  df_mod.output_text = df_mod.output_text.str.lower()
  df_mod.output_text = df_mod.output_text.str.replace('\n', NEWLINE_SPEC_TOKEN)
  df_mod.rating = df_mod.rating.apply(lambda x: 1 if x > 0 else 0)
  df_mod.genre = df_mod.genre.str.lower()
  df_mod = df_mod.drop('url', axis=1)

  return df_mod

train_data_mod = preprocess_data(train_data)
test_data_mod = preprocess_data(test_data)

# закондируем жанр через OneHotEncoder
# масштабируем просмотры через StandardScaler

ohe = OneHotEncoder()
ohe.fit(train_data_mod.genre.to_frame())

scaler = StandardScaler()
scaler.fit(train_data_mod.views.to_frame())

StandardScaler()

In [ ]:
# train_data_mod

In [ ]:
# MODEL = 'DeepPavlov/rubert-base-cased'
MODEL = 'cointegrated/rubert-tiny2'

tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.add_special_tokens({'additional_special_tokens': [NEWLINE_SPEC_TOKEN]}) # добавляем новый токен для переноса строк

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1

In [ ]:
# выбираем макс длину последовательности
lst_len_tokens = []
for text in tqdm(train_data_mod.output_text.values):
  lst_len_tokens.append(len(tokenizer.encode(text, add_special_tokens=True)))

100%|██████████| 138396/138396 [02:03<00:00, 1123.19it/s]


In [ ]:
pd.Series(lst_len_tokens).describe(percentiles=[.75, .8, .9, .95])

,0
count,138396.000000
mean,144.579959
std,109.851165
min,10.000000
50%,129.000000
75%,184.000000
80%,200.000000
90%,248.000000
95%,301.000000
max,5849.000000


In [ ]:
# MAX_LENGTH =  # DeepPavlov
# MAX_LENGTH = 344 #cointegrated 90%
MAX_LENGTH = 185 #cointegrated ~75%
# MAX_LENGTH = 210 #cointegrated посередине между 50%(178) и 75%(253)

In [ ]:
class DatasetPoetry(Dataset):
    def __init__(self
                 , texts
                 , genres
                 , views
                 , labels
                 , tokenizer
                 , max_len
                 ):
        self.texts = texts
        self.genres = genres
        self.views = views
        self.labels = labels

        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        genre = self.genres[item]
        views = self.views[item]
        label = self.labels[item]

        model_inputs = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'text': text,
            'input_ids': model_inputs['input_ids'].flatten(),
            'attention_mask': model_inputs['attention_mask'].flatten(),
            'genre': torch.tensor(genre, dtype=torch.long),
            'views': torch.tensor(views, dtype=torch.float),
            'labels': torch.tensor(label, dtype=torch.float).unsqueeze(0) # чтобы размеры labels и выходов модели совпадали
        }

In [ ]:
train_genre_ohe = ohe.transform(train_data_mod.genre.to_frame()).values
train_views_scaled = scaler.transform(train_data_mod.views.to_frame())

train_dataset = DatasetPoetry(
    train_data_mod.output_text.values
    , train_genre_ohe
    , train_views_scaled
    , train_data_mod.rating.values
    , tokenizer
    , MAX_LENGTH
)


test_genre_ohe = ohe.transform(test_data_mod.genre.to_frame()).values
test_views_scaled = scaler.transform(test_data_mod.views.to_frame())

test_dataset = DatasetPoetry(
    test_data_mod.output_text.values
    , test_genre_ohe
    , test_views_scaled
    , test_data_mod.rating.values
    , tokenizer
    , MAX_LENGTH
)

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer


class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_genres, dim_views=1, hidden_dim=128):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.bert.resize_token_embeddings(len(tokenizer)) # добавляем новый токен для переноса строк
        self.dropout = nn.Dropout(self.bert.config.hidden_dropout_prob)

        final_dim = self.bert.config.hidden_size + num_genres + dim_views
        print(f'final_dim = {final_dim}')

        self.linear = nn.Linear(final_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()


    def forward(self, input_ids, attention_mask, genres, views):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs[1]  # [CLS] token

        x = torch.cat((cls_output, genres, views), dim=1)
        x = self.dropout(x)
        x = self.linear(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.sigmoid(x)

        return x

In [ ]:
def test_epoch(model, val_loader, criterion):
    lost_list = []
    model.eval()
    for data in tqdm(val_loader):
        input_ids = data['input_ids'].to(DEVICE)
        padding_mask = data['attention_mask'].to(DEVICE)
        genres = data['genre'].to(DEVICE)
        views = data['views'].to(DEVICE)
        labels = data['labels'].to(DEVICE)

        with torch.no_grad():
            outputs = model(input_ids, padding_mask, genres, views)
            loss = criterion(outputs, labels)
            lost_list.append(loss.item())

    return np.sum(lost_list)/len(val_loader)


def train_epoch(model, train_loader, optimizer, criterion):
    loss_list = []
    model.train()
    for data in tqdm(train_loader):
        input_ids = data['input_ids'].to(DEVICE)
        padding_mask = data['attention_mask'].to(DEVICE)
        genres = data['genre'].to(DEVICE)
        views = data['views'].to(DEVICE)
        labels = data['labels'].to(DEVICE)

        optimizer.zero_grad()
        outputs = model(input_ids, padding_mask, genres, views)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())

    return np.sum(loss_list)/len(train_loader)


def train(model, train_loader, val_loader, optimizer, criterion, n_epochs):
    train_loss_list, val_loss_list = [], []
    for epoch in range(n_epochs):
        train_loss = train_epoch(model, train_loader, optimizer, criterion)
        val_loss = test_epoch(model, val_loader, criterion)

        print(f"Epoch {epoch}")
        print(f" train loss: {train_loss}")
        print(f" val loss: {val_loss}")

        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)

    plt.plot(train_loss_list, label='train')
    plt.plot(val_loss_list, label='val')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.legend()
    plt.show()

    return train_loss_list, val_loss_list

In [ ]:
print(DEVICE)
NUM_GENRES = ohe.get_feature_names_out().size


model = BERTClassifier(MODEL, NUM_GENRES)
model = model.to(DEVICE)

# заморозка всех слоев BERT, кроме 2 линеных
for param in model.bert.parameters():
    param.requires_grad = False

for param in model.linear.parameters():
    param.requires_grad = True

for param in model.linear2.parameters():
    param.requires_grad = True



optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()

n_epochs = 10
# n_epochs = 5
# n_epochs = 1

train_dataloader = DataLoader(train_dataset, batch_size=40, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=40, shuffle=True)

# train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

train_loss_list, val_loss_list = train(model, train_dataloader, test_dataloader, optimizer, criterion, n_epochs)

cuda


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

final_dim = 348


100%|██████████| 34599/34599 [02:43<00:00, 211.19it/s]


...

Epoch 3
 train loss: 0.606059142835717
 val loss: 0.6004201111424328


100%|██████████| 34599/34599 [02:39<00:00, 216.34it/s]


Epoch 4
 train loss: 0.6042438197177149
 val loss: 0.6002099280635729


  0%|          | 514/138396 [00:02<13:09, 174.69it/s]


KeyboardInterrupt: 

In [ ]:
test_dataloader2 = DataLoader(test_dataset, batch_size=40, shuffle=False)
batch = next(iter(test_dataloader))

In [ ]:
lst = np.array([])
model.eval()
with torch.no_grad():
    for batch in tqdm(test_dataloader2):
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        genre = batch['genre'].to(DEVICE)
        views = batch['views'].to(DEVICE)

        outputs = model(input_ids, attention_mask, genre, views)
        lst = np.concatenate((lst, outputs.squeeze().cpu().numpy()), axis=0)

100%|██████████| 865/865 [00:55<00:00, 15.69it/s]


In [ ]:
pd.Series((lst > 0.5).astype(int)).value_counts()

,count
0,24130
1,10469


In [ ]:
test_data_mod.rating.value_counts()

,count
rating,
0,19373
1,15226


In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score

In [ ]:
# 0.7311, 0.6801 = txt+genre+veiws + 5 epochs + frozen + 185 + linX2(128)
roc_auc_score(test_data_mod.rating, lst), accuracy_score(test_data_mod.rating, (lst > 0.5).astype(int))

(0.7311071034639887, 0.6801352640249718)

In [ ]:
import pandas as pd

# (0.6518974744622479, 0.6201335298708055)
pd.DataFrame({
    'params': ['tf-idf/logreg', 't+g+v/5e/frozen/185tok/linX2(128)'],
    'ROC-AUC': [0.6518, 0.7311],
    'Accuracy': [0.6201, 0.6801]
}).sort_values('ROC-AUC', ascending=False)

,params,ROC-AUC,Accuracy
1,t+g+v/5e/frozen/185tok/linX2(128),0.7311,0.6801
0,tf-idf/logreg,0.6518,0.6201
